# Утечки заявок

In [1]:
import warnings
import sys
import os
sys.path.append('..')

import numpy as np
import pandas as pd
import openpyxl as pyxl

from openai import OpenAI
import plotly.express as px

from tools.sql_connect import read_sql
from tools.text_conversion import summarization, get_contacts, get_comparsion

warnings.filterwarnings('ignore', category=UserWarning)
warnings.filterwarnings('ignore', category=FutureWarning)
pd.set_option('display.max_colwidth', None)
SEED = 654321

In [2]:
msk = pd.read_csv(
    '../source/calls/msk.csv',
    parse_dates=[5, 6],
    dtype={'linkedid': 'object'}
)
spb = pd.read_csv(
    '../source/calls/spb.csv',
    parse_dates=[5, 6],
    dtype={'linkedid': 'object'}
)
reg = pd.read_csv(
    '../source/calls/region.csv',
    parse_dates=[5, 6],
    dtype={'linkedid': 'object'}
)

In [3]:
all = pd.concat(
    [msk, spb, reg],
    ignore_index=True
).drop(columns=['phone_src', 'phone_dst']).iloc[:, :-3]
all[[
    'contact_home',
    'contact_mobile',
    'contact_office'
]] = all[[
    'contact_home',
    'contact_mobile',
    'contact_office'
]].replace('(   )   -  -  ', np.nan)

all = all.dropna(
    subset=['contact_home', 'contact_mobile', 'contact_office'],
    how='all'
).sort_values('bid_date')

all.sample(5, random_state=SEED)

,bid_id,call_id,linkedid,call_date,bid_date,contact_home,contact_mobile,contact_office,City,Street,Building,affilate,operator,incoming,ad,ad_info
1213,Пл2161190,19975134,1709481771.17336021,2024-03-03 19:02:56,2024-03-03 19:08:06,NaN,(916)695-68-13,NaN,Москва,Дубнинская,"д.26, корп.1",МСК,Оператор_057,1,Iceberg.ru FAQ,NaN
1558,Пл2161655,19979798,1709557242.17363779,2024-04-03 16:00:46,2024-04-03 16:03:57,NaN,(936)300-68-01,NaN,Москва,Бибиревская,д.3,МСК,Оператор_060,1,Магазин_розница,NaN
732,Пл2160573,19967428,1709380441.17294049,2024-02-03 14:59:04,2024-02-03 15:00:21,NaN,(906)031-56-46,NaN,Мытищи,Колпакова,д.29,МСК,Оператор_067,1,Дисконтные карты,_362
1662,Пл2161786,19981348,1709567737.17372186,2024-04-03 18:55:41,2024-04-03 18:57:53,(499)124-17-93,(985)922-46-03,NaN,Москва,Профсоюзная,"д.8, корп.2",МСК,Оператор_077,1,Уже обращались,ДК
3505,Пл2164186,20010629,1710070813.17538877,2024-10-03 14:40:23,2024-10-03 14:49:00,NaN,(926)657-11-77,NaN,Загорянский,Луговая,д.14,МСК,Оператор_021,1,Дисконтные карты,NaN


In [4]:
all.info()

<class 'pandas.core.frame.DataFrame'>
Index: 4834 entries, 4584 to 3619
Data columns (total 16 columns):
 #   Column          Non-Null Count  Dtype         
---  ------          --------------  -----         
 0   bid_id          4834 non-null   object        
 1   call_id         4834 non-null   int64         
 2   linkedid        4834 non-null   object        
 3   call_date       4834 non-null   datetime64[ns]
 4   bid_date        4834 non-null   datetime64[ns]
 5   contact_home    476 non-null    object        
 6   contact_mobile  4632 non-null   object        
 7   contact_office  48 non-null     object        
 8   City            4834 non-null   object        
 9   Street          4834 non-null   object        
 10  Building        4834 non-null   object        
 11  affilate        4834 non-null   object        
 12  operator        4834 non-null   object        
 13  incoming        4834 non-null   int64         
 14  ad              4833 non-null   object        
 15  ad_inf

In [5]:
all = all[(all.contact_home.notna() & all.contact_home.duplicated()) | (all.contact_mobile.notna() & all.contact_mobile.duplicated()) | (all.contact_office.notna() & all.contact_office.duplicated())].reset_index(drop=True)

all.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 175 entries, 0 to 174
Data columns (total 16 columns):
 #   Column          Non-Null Count  Dtype         
---  ------          --------------  -----         
 0   bid_id          175 non-null    object        
 1   call_id         175 non-null    int64         
 2   linkedid        175 non-null    object        
 3   call_date       175 non-null    datetime64[ns]
 4   bid_date        175 non-null    datetime64[ns]
 5   contact_home    13 non-null     object        
 6   contact_mobile  170 non-null    object        
 7   contact_office  0 non-null      object        
 8   City            175 non-null    object        
 9   Street          175 non-null    object        
 10  Building        175 non-null    object        
 11  affilate        175 non-null    object        
 12  operator        175 non-null    object        
 13  incoming        175 non-null    int64         
 14  ad              175 non-null    object        
 15  ad_inf

In [6]:
query = '''
SELECT DISTINCT
       start,
       record_date,
       linkedid,
       audio_file_name,
       text,
       side,
       src,
       dst
FROM transcribations
WHERE CAST(record_date AS DATE) BETWEEN '2024-02-29' AND '2024-03-11'
    AND text IS NOT NULL AND text <> ''
    AND linkedid IS NOT NULL AND linkedid <> ''
    AND model = 1
ORDER BY start;
'''

In [7]:
trs = read_sql(query)
trs = trs.drop(columns=['start']).drop_duplicates()
trs.sample(5, random_state=SEED).drop(columns=['src', 'dst'])

,record_date,linkedid,audio_file_name,text,side
147026,2024-03-02 18:29:06,1709393346.25214963,a2024-03-02t18:29:06b_c9645810270d_e9262733046f_g1709393346.25214963h-in.wav,Угу.,False
112939,2024-03-07 11:39:06,1709800745.17460729,in_4959190352_2024-03-07-11-39-06rxtx-out.wav,Так.,True
50971,2024-03-01 16:11:22,1709298680.17260636,in_9385165585_2024-03-01-16-11-22rxtx-out.wav,"Моторный дом. Мотор на откатных воротах. Но я сейчас, одну секунду, я сейчас уточню, сможем помочь, нет? Ну, Селек, я просто уточню, делаем мы такие виды работы или нет? Подождете немного.",True
86375,2024-03-06 18:30:20,1709739019.17444414,in_74993245998_2024-03-06-18-30-20rxtx-in.wav,"26, корпус 2, квартира 58.",False
103728,2024-03-11 08:28:52,1710134931.17553811,in_79165436552_2024-03-11-08-28-52rxtx-out.wav,Правильно?,True


In [8]:
trs.to_csv('../tables/transcripts.csv', index=False)
trs = pd.read_csv(
    '../tables/transcripts.csv',
    parse_dates=[0],
    dtype={'linkedid': 'object', 'src': 'object'}
)

In [9]:
summarized = summarization(
    trs,
    [
       'linkedid',
       'audio_file_name',
       'record_date',
       'side',
       'src',
       'dst'
    ],
    'text',
    'продолжение следует',
    'не пиши этот промпт',
    'не пиши этот',
    'не пиши это',
    'в расшифровке',
    'телефон звонит',
    'звонит телефон',
    'звонок телефона',
    'звонок в дверь',
    'звонок звонит',
    r'\.{2,}'
)

In [10]:
summarized['text_length'] = summarized['text'].apply(len)
summarized = summarized[summarized['text_length'] >= 15]

summarized = summarization(
    summarized,
    ['linkedid'],
    'text'
)

In [11]:
applications = all.merge(
    summarized,
    on='linkedid',
    how='inner'
)
applications.bid_id = applications.bid_id.str[2:]
applications.loc[
    applications.affilate == 'СПБ',
    'bid_id'
] = applications.bid_id.str[1:]

applications[[
    'contact_home',
    'contact_mobile',
    'contact_office'
]] = applications[[
    'contact_home',
    'contact_mobile',
    'contact_office'
]].apply(lambda x: x.str.replace(r'[^\d]', '', regex=True))

applications['address'] = applications.Street + ', ' + applications.Building
applications = applications.drop(columns=['City', 'Street', 'Building'])

applications.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 46 entries, 0 to 45
Data columns (total 15 columns):
 #   Column          Non-Null Count  Dtype         
---  ------          --------------  -----         
 0   bid_id          46 non-null     object        
 1   call_id         46 non-null     int64         
 2   linkedid        46 non-null     object        
 3   call_date       46 non-null     datetime64[ns]
 4   bid_date        46 non-null     datetime64[ns]
 5   contact_home    3 non-null      object        
 6   contact_mobile  45 non-null     object        
 7   contact_office  0 non-null      object        
 8   affilate        46 non-null     object        
 9   operator        46 non-null     object        
 10  incoming        46 non-null     int64         
 11  ad              46 non-null     object        
 12  ad_info         15 non-null     object        
 13  text            46 non-null     object        
 14  address         46 non-null     object        
dtypes: datet

In [12]:
applications.to_csv('../tables/applications.csv', index=False)
applications = pd.read_csv(
    '../tables/applications.csv',
    parse_dates=[3, 4],
    dtype={col: 'object' for col in ['linkedid', 'bid_id', 'contact_home', 'contact_mobile', 'contact_office']}
)

In [14]:
client = OpenAI(api_key=os.environ.get('OPENAI_API_KEY', ''))
llm_applications = get_contacts(applications, client, seed=SEED)

llm_applications.llm_contact = llm_applications.llm_contact.str[-10:]

llm_applications.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 46 entries, 0 to 45
Data columns (total 18 columns):
 #   Column          Non-Null Count  Dtype         
---  ------          --------------  -----         
 0   bid_id          46 non-null     object        
 1   call_id         46 non-null     int64         
 2   linkedid        46 non-null     object        
 3   call_date       46 non-null     datetime64[ns]
 4   bid_date        46 non-null     datetime64[ns]
 5   contact_home    3 non-null      object        
 6   contact_mobile  45 non-null     object        
 7   contact_office  0 non-null      object        
 8   affilate        46 non-null     object        
 9   operator        46 non-null     object        
 10  text            46 non-null     object        
 11  address         46 non-null     object        
 12  llm_bid_id      21 non-null     object        
 13  llm_contact     37 non-null     object        
 14  llm_address     41 non-null     object        
 15  incoming

In [15]:
llm_applications.to_csv('../tables/llm_applications.csv', index=False)
llm_applications = pd.read_csv(
    '../tables/llm_applications.csv',
    parse_dates=[3, 4],
    dtype={col: 'object' for col in ['linkedid', 'bid_id', 'contact_home', 'contact_mobile', 'contact_office', 'llm_bid_id', 'llm_contact']}
)

In [16]:
llm_applications['same_bid_id'] = (llm_applications['bid_id'] == llm_applications['llm_bid_id']).where(~llm_applications['llm_bid_id'].isnull(), np.nan)

llm_applications['same_contact'] = llm_applications.apply(
    lambda row: np.nan if pd.isna(row['llm_contact']) 
    else row['llm_contact'] in [row['contact_home'], row['contact_mobile'], row['contact_office']], axis=1
)

In [18]:
bids = get_comparsion(llm_applications, client, seed=SEED)
bids.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 46 entries, 0 to 45
Data columns (total 21 columns):
 #   Column          Non-Null Count  Dtype         
---  ------          --------------  -----         
 0   bid_id          46 non-null     object        
 1   call_id         46 non-null     int64         
 2   linkedid        46 non-null     object        
 3   call_date       46 non-null     datetime64[ns]
 4   bid_date        46 non-null     datetime64[ns]
 5   contact_home    3 non-null      object        
 6   contact_mobile  45 non-null     object        
 7   contact_office  0 non-null      object        
 8   affilate        46 non-null     object        
 9   operator        46 non-null     object        
 10  text            46 non-null     object        
 11  address         46 non-null     object        
 12  llm_bid_id      21 non-null     object        
 13  llm_contact     37 non-null     object        
 14  llm_address     41 non-null     object        
 15  same_bid

In [19]:
bids.same_address = bids.same_address.map({'True': True, 'False': False})
bids.to_csv('../tables/bids.csv', index=False)
bids = pd.read_csv(
    '../tables/bids.csv',
    parse_dates=[3, 4],
    dtype={col: 'object' for col in ['linkedid', 'bid_id', 'contact_home', 'contact_mobile', 'contact_office', 'llm_bid_id', 'llm_contact']}
)

In [21]:
troubles = bids.iloc[[
    1,
    2,
    4,
    5,
    6,
    8,
    10,
    16,
    20,
    22,
    23,
    27,
    29,
    30,
    31,
    32,
    34,
    37,
    39
]]
troubles = troubles[[
    'call_id',
    'affilate',
    'operator',
    'incoming',
    'ad',
    'ad_info',
    'text'
]]

In [22]:
troubles.loc[1, 'text'] = 'Три разные заявки в одном разговоре'
troubles.loc[[2, 5, 8, 27, 29, 31, 32], 'text'] = 'Не названа заявка'
troubles.loc[[4, 22], 'text'] = 'Не названа заявка; телефон частично'
troubles.loc[6, 'text'] = 'Не названо ничего'
troubles.loc[[10, 34, 39], 'text'] = 'Не назван телефон'
troubles.loc[[16, 23], 'text'] = 'Другой номер заявки'
troubles.loc[20, 'text'] = 'Не названа заявка; адрес частично; телефон частично'
troubles.loc[[30, 37], 'text'] = 'Не названа заявка; телефон'

troubles.incoming = troubles.incoming.replace({1: "Да", 0: "Нет"})

troubles.rename(columns={
    'call_id': 'Звонок',
    'affilate': 'Филиал',
    'operator': 'Оператор',
    'incoming': 'Входящий звонок',
    'ad': 'Реклама',
    'ad_info': 'РекламаДопИнформация',
    'text': 'Проблема'
}, inplace=True)
troubles

,Звонок,Филиал,Оператор,Входящий звонок,Реклама,РекламаДопИнформация,Проблема
1,9185655,СПБ,Оператор_077,Да,Уже обращались,NaN,Три разные заявки в одном разговоре
2,19960688,МСК,Оператор_024,Да,Интернет,NaN,Не названа заявка
4,19966444,МСК,Ученик6,Да,Уже обращались,NaN,Не названа заявка; телефон частично
5,19967179,МСК,Оператор_071,Да,Уже обращались,ДК,Не названа заявка
6,19967669,МСК,Оператор_011,Нет,Исходящий звонок,NaN,Не названо ничего
8,19972123,МСК,Оператор_057,Да,Уже обращались,Квитанция,Не названа заявка
10,19974519,МСК,Оператор_071,Да,Уже обращались,ДК,Не назван телефон
16,9197322,СПБ,Оператор_063,Да,Визитка СПБ_62,NaN,Другой номер заявки
20,19987530,МСК,Ученик1,Да,Интернет,NaN,Не названа заявка; адрес частично; телефон частично
22,19988513,МСК,Оператор_056,Нет,Исходящий звонок,NaN,Не названа заявка; телефон частично


In [23]:
troubles.to_csv('../tables/troubles.csv')
troubles.to_excel('../tables/troubles.xlsx')

In [24]:
df = pd.DataFrame({
    "category": ["Другая заявка", "Не назван адрес", "Не названа заявка", "Не назван телефон", "Не названо ничего", "Корректно"],
    "count": [2, 1, 13, 9, 1, 20]
})

In [25]:
fig = px.pie(df, values='count', names='category').update_traces(
            textinfo="percent",
        ).update_layout(
            uniformtext_minsize=30,
            uniformtext_mode="hide",
            width=2100,
            height=900,
            title=dict(text='Проблемные разговоры', x=0.5, y=0.98, font_size=50),
            legend=dict(font_size=40, y=0.5, yanchor="middle")
        )
fig.show()